# Ejercicio 1

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import glob
import os

plt.rcParams['figure.figsize'] = (12, 6)
plt.style.use('seaborn')

In [4]:
dir_actual = os.getcwd()
archivos_dump = list(map(lambda x: f'{dir_actual}/{x}', glob.glob('input/dump/*.csv')))

dfs = (pd.read_csv(f) for f in archivos_dump)
df = pd.concat(dfs).drop(columns='Unnamed: 0')
df.columns = ['artist', 'genre', 'song_name', 'lyrics']

# Ejercicio 2

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english', max_features=5000)
vectorizer_fit = vectorizer.fit_transform(df['lyrics'])

words = vectorizer.get_feature_names_out()
freqs = vectorizer_fit.toarray().sum(axis=0)

print(f'Las 5000 palabras mas frecuentes en la base de datos son: {words}')

Las 5000 palabras mas frecuentes en la base de datos son: ['000' '10' '100' ... 'zoo' 'zoom' 'zulu']


# Ejercicio 3

In [6]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

In [14]:
n_components = [5, 10, 15]
learning_decay = [0.7, 0.5]

grid = {
    'n_components': n_components,
    'learning_decay': learning_decay
}

modelo_grilla = GridSearchCV(LatentDirichletAllocation(learning_method='online'), 
                             param_grid=grid, verbose=5, cv=5)

In [16]:
X = CountVectorizer(stop_words='english', max_features=5000, max_df=0.1).fit_transform(df['lyrics'].values)

In [17]:
modelo_grilla.fit(X)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END learning_decay=0.7, n_components=5;, score=-1163788.469 total time=  35.0s
[CV 2/5] END learning_decay=0.7, n_components=5;, score=-1473371.251 total time=  33.3s
[CV 3/5] END learning_decay=0.7, n_components=5;, score=-1321940.552 total time=  34.2s
[CV 4/5] END learning_decay=0.7, n_components=5;, score=-1171146.482 total time=  32.0s
[CV 5/5] END learning_decay=0.7, n_components=5;, score=-1475996.457 total time=  30.5s
[CV 1/5] END learning_decay=0.7, n_components=10;, score=-1180431.711 total time=  35.4s
[CV 2/5] END learning_decay=0.7, n_components=10;, score=-1488246.542 total time=  35.0s
[CV 3/5] END learning_decay=0.7, n_components=10;, score=-1341744.658 total time=  34.5s
[CV 4/5] END learning_decay=0.7, n_components=10;, score=-1192856.468 total time=  34.9s
[CV 5/5] END learning_decay=0.7, n_components=10;, score=-1493181.077 total time=  32.2s
[CV 1/5] END learning_decay=0.7, n_components=15;, scor

GridSearchCV(cv=5,
             estimator=LatentDirichletAllocation(learning_method='online'),
             param_grid={'learning_decay': [0.7, 0.5],
                         'n_components': [5, 10, 15]},
             verbose=5)

In [19]:
modelo_grilla.best_params_

{'learning_decay': 0.7, 'n_components': 5}

In [20]:
mejor_learning_decay = 0.7
mejor_n_components = 5
best_lda = LatentDirichletAllocation(n_components=mejor_n_components, 
                                     learning_decay=mejor_learning_decay,
                                     learning_method='online')
best_lda.fit(X) 

LatentDirichletAllocation(learning_method='online', n_components=5)

In [28]:
for topic_id, topic_name in enumerate(best_lda.components_):
# para cada tópico
    print("tópico: {}".format(topic_id + 1))
    print(" ".join([vectorizer.get_feature_names_out()[i] for i in
    topic_name.argsort()[:-15 - 1: -1]]))

tópico: 1
niggas umm niggaz lyrics rap funk goons wiz bird garbage humanity cheat beans vibrations chopped
tópico: 2
ones ah ladies tony rock example yesterday dang whores nah hail roll round daisy ba
tópico: 3
mall fuse funky creation damn froze helps shadow helped empire boats hatred tool buyin alot
tópico: 4
goodness dies workout west thousand benz remind tan gun maybe young tiny knot looked heartbeat
tópico: 5
pages sunrise familiar dressed bein losing forsake slang fresh drill fee moments alien darling figures
